Modelo Conceitual
<img src="https://github.com/gtabchoury/trabalhoMC536/blob/master/documentos/Modelo%20Conceitual.png"><img>

Modelo Lógico

PESSOA (id, idade, genero, estado, idEmpresa)
idEmpresa-> Chave Estrangeira: Empresa(id)
estado> Chave Estrangeira: Estado(sigla)

PESSOA_RESPOSTA (id, idPessoa, data_hora, busca_tratamento, interfere_trabalho, conhece_opcoes, programa_saude, oferece_recursos, anonimidade, licenca_medica, mental_consequencia, fisica_consequencia, colegas_trabalho, supervisor, mental_entrevista, fisica_entrevista, mental_fisica, houve_consequencias)
idPessoa -> Chave Estrangeira: Pessoa(id)

EMPRESA(id, n_funcionarios, area_atuacao, beneficios)

ESTADO(sigla, pais)
pais> Chave Estrangeira: Pais(nome)

PAIS(nome)

In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
DROP TABLE IF EXISTS Pessoa;
CREATE TABLE Pessoa (
  id VARCHAR(10) NOT NULL,
  idade SMALLINT,
  genero VARCHAR(20),
  pais VARCHAR(50),
  estado VARCHAR(10),
  PRIMARY KEY(id),
) AS SELECT id, idade, genero, pais, estado 
FROM CSVREAD('pessoa.csv');

DROP TABLE IF EXISTS Pessoa_Resposta;
CREATE TABLE Pessoa_Resposta (
  idPessoa VARCHAR(10) NOT NULL,
  data_hora Timestamp,
  trabalho_proprio VARCHAR(10),
  historico_familiar VARCHAR(10),
  tratamento VARCHAR(10),
  interferencia_trabalho VARCHAR(10),
  trabalho_remoto VARCHAR(10),
  empresa_ti VARCHAR(10),
  conhece_opcoes VARCHAR(10),
  programa_saude VARCHAR(10),
  busca_tratamento VARCHAR(10),
  anonimidade VARCHAR(10),
  licenca_medica VARCHAR(20),
  mental_consequencia VARCHAR(10),
  fisica_consequencia VARCHAR(10),
  mental_entrevista VARCHAR(10),
  fisica_entrevista VARCHAR(10),
  mental_fisica VARCHAR(10),
  houve_consequencias VARCHAR(10),
  PRIMARY KEY(idPessoa),
  FOREIGN KEY(idPessoa)
    REFERENCES Pessoa(id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
) AS SELECT idPessoa, data_hora, trabalho_proprio, historico_familiar, tratamento, interferencia_trabalho, trabalho_remoto, 
empresa_ti, conhece_opcoes, programa_saude, busca_tratamento, anonimidade, licenca_medica, mental_consequencia, fisica_consequencia,
mental_entrevista, fisica_entrevista, mental_fisica, houve_consequencias 
FROM CSVREAD('resposta.csv');


In [10]:
delete from Pessoa_Resposta where idPessoa = 388;
delete from Pessoa where id = 388

// Selecionar a quantidade de entrevistados por genero

In [11]:
SELECT P.genero, COUNT(*)
FROM Pessoa_Resposta R, Pessoa P 
WHERE R.idPessoa=P.id  
GROUP BY P.genero 
ORDER BY genero

// Selecionar a interferencia do trabalho por genero

In [12]:
SELECT genero, COUNT(*) quantidade
FROM Pessoa, Pessoa_Resposta
WHERE idPessoa=id 
AND (interferencia_trabalho='Sometimes' OR interferencia_trabalho='Often') 
GROUP BY genero

In [5]:
SELECT T.genero, T.total, I.quantidade, ROUND((CAST(I.quantidade as double)/CAST(T.total as double))*100,1) porcentagem
FROM 
    (SELECT genero, COUNT(*) total
    FROM Pessoa_Resposta, Pessoa 
    WHERE idPessoa=id  
    GROUP BY genero) T,
    (SELECT genero, COUNT(*) quantidade
    FROM Pessoa, Pessoa_Resposta
    WHERE idPessoa=id 
    AND (interferencia_trabalho='Sometimes' OR interferencia_trabalho='Often') 
    GROUP BY genero) I
WHERE T.genero = I.genero
    

// Descobrir se pessoas que cuidam do próprio negócio buscam mais tratamento que as outras

In [26]:
SELECT trabalho_proprio, tratamento,  count(*) 
FROM Pessoa_Resposta
WHERE trabalho_proprio <> 'NA'
GROUP BY trabalho_proprio, tratamento
ORDER BY trabalho_proprio

In [6]:
// Selecionar a interferencia do trabalho por por idade

In [7]:
SELECT P.idade, COUNT(*) FROM Pessoa_Resposta R, Pessoa P where R.idPessoa=P.id and (R.interferencia_trabalho='Sometimes' OR R.interferencia_trabalho='Often') GROUP BY P.idade

// Selecionar a quantidade de pessoas que buscam tratamento por idade

In [8]:
SELECT P.idade, COUNT(*) FROM Pessoa_Resposta R, Pessoa P where R.idPessoa=P.id and R.busca_tratamento='Yes' GROUP BY P.idade

// Selecionar a quantidade de pessoas que buscam tratamento por genero

In [9]:
SELECT P.genero, COUNT(*) FROM Pessoa_Resposta R, Pessoa P where R.idPessoa=P.id and R.busca_tratamento='Yes' GROUP BY P.genero